### 관찰
- 2년 전에 풀었던 문제의 강화판이다. 우선 11066(파일 합치기)의 풀이를 다시 발상해보자.
  - 어떤 파일을 합치면, 그 파일의 크기와, 그 파일을 만드는데에 필요했던 누적 합치는 비용 끼리를 더하는 것과 같다.
  - 즉, 임의의 두 원소를 합치고, 그 과정에서 나오는 최소 비용의 합을 DP에 저장시키는 과정을 $N-1$ 회 반복하면 된다.
  - ...인줄 알았는데 문제 조건에 `연속하는 파일을` 합쳐야 한다고 적혀있었다. 저번에도 이걸 잊었었던거 같은데, 배열의 위치를 바꿀 수 있다는 내용을 또 안 읽고 말았다.\
  애초에 생각했던 풀이대로라면, 정렬을 하거나, 매번 합친 비용과 누적된 합칠 비용이 가장 작은 것만 매번 골라서 더 빠르게 풀 수 있다. 그것이 13975(파일 합치기 3)의 풀이이기도 하고..\
  어쨋든 이러한 풀이도 $O(N^3)$ 일 것이다. 매번 sequence 원소 중 2개를 고르는 경우는, 현재의 sequence 길이가 $n$ 이라고 할때, $\binom{n}{2}$ 이므로 $O(N^2)$ 이다.\
  이 과정을 $N-1$ 번 반복하므로, 총 $O(N^3)$ 이다.
  - 정해는, 누적합을 구해놓고 구간의 길이 $d = 1 \to k$ 에 따라, $DP[i][j] = [i, j) \text{ 까지 파일을 합쳤을 때 최솟값}$ 을 갱신하는 것이다.\
  마찬가지로 시간복잡도는 $O(N^3)$ 이겠지만, 연산량이 절반 보다 조금 많을 것이고, 연속하게 합치는 경우만 고려할 것이다.
    - $DP[i][j] = \min_{i < k < j} DP[i][k] + DP[k][j] + \sum_{i \leq l < j} L[l]$
  - sequence가 주워지는 문제에선 반드시 순서를 바꿔도 되는지 먼저 확인하는 습관을 가져보자.
    - 구간합, 인접한, 연속하는 등의 단어에 민감하게 반응해야 할 것이다.



### 아이디어


### 더 알아보기
- 참고: https://tistory.joonhyung.xyz/15 
  - 13975(파일 합치기 3)은 Huffman coding, 1,2,4는 alphabetical Huffman coding 혹은 Hu-Tucker coding이라고 한다.
  - 이걸 $O(N \log N)$ 에 푸는 알고리즘을 Garsia-Wachs algorithm 이라고 한다.